In [1]:
import random
random.seed(123)

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# importing feature selection and processing packages

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import MinMaxScaler,StandardScaler,PowerTransformer
from sklearn.decomposition import PCA

# importing modelling packages

from sklearn.linear_model import LogisticRegression, RidgeClassifier

from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv('train_final.csv')
test = pd.read_csv('test_final.csv')

In [3]:
train_data = train.copy()
test_data = test.copy()

In [4]:
id = test_data["Id"].values
train_data.drop('Id', axis=1, inplace=True)
test_data.drop('Id', axis=1, inplace=True)
train_data.head(5)
test_data.head(5)

,2,3,4,5,6,7,8,9,10,11,...,375,376,377,378,379,380,381,382,383,384
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,2175.0,0,1,1,0.195686,-0.694456
1,0,1,0,0,0,1,0,0,1,0,...,0,1,0,1,5667.0,0,1,6,-0.789394,1.159170
2,0,0,70,0,0,0,0,0,1,0,...,0,0,0,1,3498.0,0,1,3,-2.717848,-1.179772
3,0,0,0,0,0,1,0,0,1,0,...,0,0,0,4,4465.0,0,1,4,1.953888,0.756751
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,13,-3968.0,0,1,1,5.102421,-2.573633


In [5]:
# splitting data 

X = train_data.drop('Y',axis=1)
y = train_data['Y'] # the target variable

In [6]:
# using baseline xgb and catboost models - on gpu



In [7]:
test_for_model = test_data

In [8]:
# 10 splits
folds = StratifiedKFold(n_splits = 10, random_state = 2021, shuffle = True)

predictions_cb = np.zeros(len(test_for_model))

cat_oof = np.zeros(X.shape[0])

for fold, (trn_idx, val_idx) in enumerate(folds.split(X,y)):
    print(f"Fold: {fold+1}")
    X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]

    model_cb =  CatBoostClassifier(verbose=0, learning_rate = 0.01, depth = 11)

    model_cb.fit(X_train, y_train)
    pred_cb = model_cb.predict_proba(X_val)[:,1]
    cat_oof[val_idx] = pred_cb
    print('ROC of CB: ',roc_auc_score(y_val,pred_cb))
    
    print("-"*50)
    
    predictions_cb += model_cb.predict_proba(test_for_model)[:,1] / folds.n_splits

Fold: 1
ROC of CB:  0.8658909461525395
--------------------------------------------------
Fold: 2
ROC of CB:  0.8817754656821615
--------------------------------------------------
Fold: 3
ROC of CB:  0.8680986742742071
--------------------------------------------------
Fold: 4
ROC of CB:  0.9137077500853533
--------------------------------------------------
Fold: 5
ROC of CB:  0.8441447593035165
--------------------------------------------------
Fold: 6
ROC of CB:  0.9051724137931034
--------------------------------------------------
Fold: 7
ROC of CB:  0.8092352338682144
--------------------------------------------------
Fold: 8
ROC of CB:  0.8859679071355411
--------------------------------------------------
Fold: 9
ROC of CB:  0.8405599180607716
--------------------------------------------------
Fold: 10
ROC of CB:  0.906452714236941
--------------------------------------------------


In [9]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(model_cb, X, y)
print ("Cross validation score for catboost: ",np.mean(score)*100)

Cross validation score for catboost:  82.86645504207884


In [10]:
sub_proba = model_cb.predict_proba(test)
pred_col=np.c_[id,sub_proba[:,1]]

submission = pd.DataFrame(pred_col, columns = ['Id','Y'])
submission["Id"] = submission["Id"].astype("Int32")
submission

,Id,Y
0,2604,0.023073
1,2605,0.081488
2,2606,0.067801
3,2607,0.015555
4,2608,0.724958
...,...,...
3599,6203,0.123549
3600,6204,0.061912
3601,6205,0.021502
3602,6206,0.061358


In [11]:
submission.to_csv('lastday1submission.csv', index=False)

In [12]:
Cross validation score for catboost:  82.86645504207884

SyntaxError: invalid syntax (3783389034.py, line 1)